In [2]:
using Flux
using Flux: onehot, crossentropy, throttle, argmax
using Flux.Data: Tree, children, isleaf

In [3]:
gpu(x) = x
# using CuArrays
# gpu(x) = cu(x)

gpu (generic function with 1 method)

## Recursive Networks

This notebook is about tree-structured data, such as parse trees used to represent sentences of english.

Here's an example of a tree, containing letters of the string 'abc'.

In [4]:
('a', ('b', 'c'))

('a', ('b', 'c'))

The tree has two branches; one ends in `'a'`, and the other is another tree containing `'b'` and `'c'`. In a neural network, we would represent the characters as numeric arrays, perhaps by one-hot encoding them.

In [5]:
alphabet = ['a', 'b', 'c']
a, b, c = [onehot(char, alphabet) for char in ('a', 'b', 'c')]
tree = (a, (b, c))

(Bool[true, false, false], (Bool[false, true, false], Bool[false, false, true]))

Simple neural networks deal with data of *fixed size*, like images. But more complex models that deal with language must handle both very long and very short sentences, meaning different sizes and shapes of trees.

How can we deal with trees of any size? For simplicity, let's just look at the sub-tree `(b, c)`. One way to simplify the problem is to combine `b` and `c` into a single vector of length 3.

In [6]:
shrink = Dense(6, 3)
combine(x, y) = shrink([a; b])
combine(b, c)

Tracked 3-element Array{Float64,1}:
 -0.084052 
  0.383361 
  0.0908063

We've effectively compressed the tree `(b, c)` into a simpler object that looks a bit like a character encoding; but it stores information about a tree, rather than a single character. Let's put this back into our original tree.

In [7]:
bc = combine(b, c)
(a, bc)

(Bool[true, false, false], param([-0.084052, 0.383361, 0.0908063]))

You might notice that our original tree now looks a lot simpler, as it only has two leaves -- and because of that, we can apply the same operation again!

In [8]:
combine(a, bc)

Tracked 3-element Array{Float64,1}:
 -0.084052 
  0.383361 
  0.0908063

We've reduce our tree to a simple vector. You can see how the same logic might apply to trees of any size or shape; indeed, we can create a function to do exactly that, called a *recursive neural network*.

In [9]:
net(a) = a
net(a::Tuple) = combine(net(a[1]), net(a[2]))

net(tree)

Tracked 3-element Array{Float64,1}:
 -0.084052 
  0.383361 
  0.0908063

The `net` function walks over the tree and does what we did above automatically. It works over trees of any shape.

In [10]:
net(((b, c), (a, (c, b))))

Tracked 3-element Array{Float64,1}:
 -0.084052 
  0.383361 
  0.0908063

Now we just need to map the tree description created by `net` into a useful form. For example, we might want to carry out a sentiment analysis and ask whether `'abc'` sounds positive or negative. In that case we can follow the recursive net with a simple two-class logistic regression as usual.

In [11]:
model = Chain(net, Dense(3, 2), softmax)
model(tree)

Tracked 2-element Array{Float64,1}:
 0.453628
 0.546372

## The Stanford Sentiment Treebank

In [12]:
include("data.jl");

The Stanford Sentiment Treebank stores parse trees for sentences scraped from movie reviews, like the following. Each sub-tree is assigned a sentiment score from 1 to 5 marking how positive it is. Scores can very a lot within the tree, and a model needs to show sensitivity to things like negation; for example, a tree like `("not", X)` will probably have the opposite sentiment of `X` (consider `X = ("very", "good")`).

In [13]:
traintrees[1]

Tree{Any}
(3, nothing)
├─ (2, nothing)
│  ├─ (2, "The")
│  └─ (2, "Rock")
└─ (4, nothing)
   ├─ (3, nothing)
   │  ├─ (2, "is")
   │  └─ (4, nothing)
   │     ├─ (2, "destined")
   │     └─ (2, nothing)
   │        ├─ (2, nothing)
   │        │  ├─ (2, nothing)
   │        │  │  ├─ (2, nothing)
   │        │  │  │  ├─ (2, "to")
   │        │  │  │  └─ (2, nothing)
   │        │  │  │     ├─ (2, "be")
   │        │  │  │     └─ (2, nothing)
   │        │  │  │        ├─ (2, "the")
   │        │  │  │        └─ (2, nothing)
   │        │  │  │           ├─ (2, "21st")
   │        │  │  │           └─ (2, nothing)
   │        │  │  │              ├─ (2, nothing)
   │        │  │  │              │  ├─ (2, "Century")
   │        │  │  │              │  └─ (2, "'s")
   │        │  │  │              └─ (2, nothing)
   │        │  │  │                 ├─ (3, "new")
   │        │  │  │                 └─ (2, nothing)
   │        │  │  │                    ├─ (2, "``")
   │        │  │  │       

We'll one-hot-encode each word that comes up, with respect to all other words in the training set.

In [14]:
really = onehot("really", alphabet)

8737-element Flux.OneHotVector:
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false

Here's the sentence above encoded this way.

In [15]:
train[1]

Tree{Any}
(nothing, Bool[false, false, false, true, false])
├─ (nothing, Bool[false, false, true, false, false])
│  ├─ (Bool[true, false, false, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false], Bool[false, false, true, false, false])
│  └─ (Bool[false, true, false, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false], Bool[false, false, true, false, false])
└─ (nothing, Bool[false, false, false, false, true])
   ├─ (nothing, Bool[false, false, false, true, false])
   │  ├─ (Bool[false, false, true, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false], Bool[false, false, true, false, false])
   │  └─ (nothing, Bool[false, false, false, false, true])
   │     ├─ (Bool[false, false, false, true, false, false, false, false, false, false  …  false, false, false, false, false, false,

To avoid working with 8,000-dimensional vectors, we'll immediately multiple by an *embedding matrix* that simply compresses the word vector.

In [16]:
N = 300
embedding = gpu(param(randn(N, length(alphabet))))

really = embedding*really

Tracked 300-element Array{Float64,1}:
  2.20154  
  1.6198   
  1.57271  
  1.22533  
  1.04339  
 -0.241978 
  1.23692  
 -0.867517 
 -1.3183   
 -0.830677 
 -0.308866 
 -1.10678  
  0.509582 
  ⋮        
  2.643    
 -0.353496 
  0.840325 
  0.297648 
 -0.817481 
 -0.0541709
  0.860149 
 -0.260532 
 -0.654112 
  1.04239  
  0.337328 
  1.00233  

We need a way to combine tokens / phrases, just as in the original example.

In [17]:
W = gpu(Dense(2N, N, relu))
combine(a, b) = W([a; b])

combine(really, really)

Tracked 300-element Array{Float64,1}:
 0.0      
 0.0      
 0.0      
 0.0      
 0.739153 
 0.0      
 0.0      
 0.85845  
 0.0      
 1.24181  
 0.0      
 0.493799 
 0.0      
 ⋮        
 1.32019  
 0.364528 
 2.12723  
 1.09511  
 1.37244  
 0.0693119
 0.616656 
 0.0      
 1.67847  
 0.817054 
 0.0      
 0.0      

Once we've got the embedding (300D vector) for a word or phrase, we want to analyse its sentiment. We'll do that with a simple linear layer.

In [18]:
sentiment = gpu(Chain(Dense(N, 5), softmax))

sentiment(really)

Tracked 5-element Array{Float64,1}:
 0.0191866
 0.190934 
 0.571709 
 0.166764 
 0.0514065

Now we can define our forward pass. It's a little more complex than the original example; because we want to predict the sentiment for each sub-tree, we need to carry the loss forward and sum it over the whole tree. So `forward` returns both an embedding an a loss for each subtree.

In [19]:
function forward(tree)
  if isleaf(tree)
    token, sent = tree.value
    phrase = embedding * token
    phrase, crossentropy(sentiment(phrase), gpu(collect(sent)))
  else
    _, sent = tree.value
    c1, l1 = forward(tree[1])
    c2, l2 = forward(tree[2])
    phrase = combine(c1, c2)
    phrase, l1 + l2 + crossentropy(sentiment(phrase), gpu(collect(sent)))
  end
end

loss(tree) = forward(map(gpu, tree))[2]

loss(train[1])

140.9116878934411 (tracked)

In [20]:
opt = ADAM(params(embedding, W, sentiment))
evalcb = () -> @show loss(train[1])

(::#13) (generic function with 1 method)

In [ ]:
Flux.train!(loss, zip(train), opt,
            cb = throttle(evalcb, 10))

Our prediction function is very simple, modelled on the original example.

In [ ]:
phrase(x) = embedding*onehot(x, alphabet)
phrase(x::Tuple) = combine(phrase(x[1]), phrase(x[2]))
predict = Chain(phrase, sentiment, argmax)

As predicted, the network is able to learn that "not" negates, and can match our intuition about how positive or negative a phrase sounds.

In [ ]:
predict(("very", "good"))

In [ ]:
predict(("not", ("very", "good")))

In [ ]:
predict(("utterly", "awful"))

In [ ]:
predict(("not", ("utterly", "awful")))